## IST664 - Homework 2

Originality assertion: All of the text and comments in this file are my original work (except for template items written by the instructor). All of the code in this file is my work, except where I give credit to another source. By adding my name below, I affirm this originality assertion.

## My name: _Darrell_Collison_ #

**Task 1: Use Beautiful Soup**

In [1]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Beautiful Soup for its web scraping capabilities
import bs4 as bs
import urllib.request # For retrieving from web pages
import re # Regular expressions
import spacy

In [3]:
# References: https://stackoverflow.com/questions/16627227/how-do-i-avoid-http-error-403-when-web-scraping-with-python

wiki_url_1 = "https://en.wikipedia.org/wiki/Darren_Collison"
wiki_url_2 = "https://en.wikipedia.org/wiki/Dimebag_Darrell"


# Define custom headers with a User-Agent string
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Create a Request object with the URL and headers
req = urllib.request.Request(url=wiki_url_1, headers=headers)
req2 = urllib.request.Request(url=wiki_url_2, headers=headers)

# 4. Open the URL using the Request object
try:
    scraped_data = urllib.request.urlopen(req)
    print("Connection successful! Status code:", scraped_data.getcode())

except urllib.error.HTTPError as e:
    print(f"Failed to open URL: {e.code} - {e.reason}")
    print("The error page content:", e.fp.read().decode('utf-8')) # Read the error message provided by the 
    
type(scraped_data) # A response object for a web page



Connection successful! Status code: 200


http.client.HTTPResponse

In [4]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Now extract the text from the article and organize into paragraphs
article = scraped_data.read() # Extract the data from the response object

parsed_article = bs.BeautifulSoup(article,'lxml') # Use lxml as the back end parser

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

len(article_text)

7026

**Task 2: Use RegEx to remove Wikipedia References and Extra Space**

In [6]:
# Put your code for task 2 here

clean_text = re.sub(r'\[[0-9]*\]', '', article_text)
cleaner_text = re.sub( r'\s+', ' ', clean_text)

print(cleaner_text)

 Darren Michael Collison (born August 23, 1987) is an American former professional basketball player who played in the National Basketball Association (NBA). Collison played four seasons of college basketball for the UCLA Bruins. He earned All-Pac-10 conference honors three times, and won the Frances Pomeroy Naismith Award his senior year as the top college player standing 6 feet 0 inches (1.83 m) or under. He was drafted by the New Orleans Hornets in the first round with the 21st overall pick of the 2009 NBA draft. Collison also played for the Indiana Pacers, Dallas Mavericks, Los Angeles Clippers, Sacramento Kings, and Los Angeles Lakers. Collison was born in Rancho Cucamonga, California to parents Dennis and June Griffith, who were both elite track and field athletes for Guyana. As a senior at Etiwanda High School under coach Dave Kleckner, Collison was named a fourth-team Parade All-American. Considered a four-star recruit by Rivals.com, Collison was listed as the No. 16 point guar

**Task 3: Tokenize with spaCy**

In [7]:
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

my_article = nlp(cleaner_text)

len(my_article) # Length in tokens

1405

In [8]:
# [token.text for token in my_article]

In [9]:
# Here's one way to work with individual sentences:
my_spans = list(my_article.sents)

my_spans[1] # Let's view just the first sentence

Collison played four seasons of college basketball for the UCLA Bruins.

In [10]:
len(my_spans)

58

**Task 4: Use displaCy to show named entities for an early sentence**

In [11]:
# Put your code for task 4 here
from spacy import displacy
displacy.render(my_article, style='ent', jupyter=True, options={'distance': 90})

For the following tasks, create a pandas dataframe and store each disovered token in an appropriately-named column.

**Task 5: Find the Root Verbs for each span**

Make a sentence by sentence list of all of the root verbs.

In [12]:
my_spans[1].root # The span object has an attribute that points to the root token

played

In [13]:
[span.root for span in my_spans]

[is,
 played,
 earned,
 drafted,
 played,
 born,
 named,
 listed,
 was,
 awarded,
 averaged,
 averaged,
 returned,
 named,
 led,
 won,
 named,
 finished,
 was,
 considered,
 selected,
 became,
 handed,
 broke,
 became,
 finished,
 traded,
 traded,
 became,
 was,
 came,
 missed,
 started,
 regained,
 signed,
 backed,
 started,
 called,
 overcame,
 signed,
 offered,
 had,
 ruled,
 suspended,
 signed,
 had,
 led,
 had,
 ended,
 announced,
 cited,
 expected,
 came,
 signed,
 is,
 were,
 represented,
 is]

In [14]:
row_root2 =[]
for span in my_spans: # Iterate through the tokens - will work for any length document
    row = span.root
    row_root2.append(row) # Append the row to the existing rows
    

In [15]:
print(row_root2)
len(row_root2)

[is, played, earned, drafted, played, born, named, listed, was, awarded, averaged, averaged, returned, named, led, won, named, finished, was, considered, selected, became, handed, broke, became, finished, traded, traded, became, was, came, missed, started, regained, signed, backed, started, called, overcame, signed, offered, had, ruled, suspended, signed, had, led, had, ended, announced, cited, expected, came, signed, is, were, represented, is]


58

**Task 6: Find the Subjects of Each Span**

Put a column in your pandas dataframe and fill it with the subjects from each span.

In [16]:
# Here's one simple way to find the subject of a sentence
for tok in my_spans[1]:
  if tok.dep_ == "nsubj":
    print(tok)

Collison


In [17]:
from tabulate import tabulate

tags = [(i, i.dep_) for i in my_spans[1]]
print(tabulate(tags, headers=["Token", "POS Tag"]))

Token       POS Tag
----------  ---------
Collison    nsubj
played      ROOT
four        nummod
seasons     dobj
of          prep
college     compound
basketball  pobj
for         prep
the         det
UCLA        compound
Bruins      pobj
.           punct


In [18]:
row_sub =[]
for span in my_spans:
    for tok in span:
        if tok.dep_ == "nsubj":
            row_sub.append(tok.text)
            break

In [19]:
# print(row_sub)
len(row_sub)

46

**Task 7: Find the Direct Objects of Each Span**

Put a column in your pandas dataframe and fill it with the direct objects from each span.

In [20]:
for tok in my_spans[1]:
  if tok.dep_ == "dobj":
    print(tok)

seasons


In [21]:
row_obj =[]
for span in my_spans:
    for tok in span:
        if tok.dep_ == "dobj":
            row_obj.append(tok.text)
            break
        

In [22]:
# print(row_obj)
len(row_obj)

39

**Task 8: Find the first Named Entity (if any) from Each Span**

Put two columns in your pandas dataframe. Find the first named entity in a span (if any) and record the ent_type_ and the corresponding token in those two columns. 

In [23]:
for tok in my_spans[1]:
  if len(tok.ent_type_) > 0:
    print(tok.ent_type_, tok)

PERSON Collison
DATE four
DATE seasons


In [24]:
row_ent =[]
row_tok =[]

for span in my_spans:
    for tok in span:
        if len(tok.ent_type_) > 0:
            row_ent.append(tok.ent_type_)
            row_tok.append(tok.text)


In [25]:
# print(row_tok)

**Task 9: Show the shape of your data frame**

In [26]:
# Add code to show the shape of your data frame

In [27]:
import pandas as pd # Use a pandas DF
cols = ("Verbs", "Subjects","Objects", "Entity", "Ent_Type")
rows = [] # A blank list to hold the rows

row = [row_root2, row_sub, row_obj, row_tok, row_ent]
rows.append(row) # Append the row to the existing rows

my_df = pd.DataFrame(rows, columns=cols)

my_df

,Verbs,Subjects,Objects,Entity,Ent_Type
0,"[is, played, earned, drafted, played, born, na...","[Collison, Collison, He, Collison, who, He, Co...","[seasons, honors, American, recruit, MVP, stea...","[Darren, Michael, Collison, August, 23, ,, 198...","[PERSON, PERSON, PERSON, DATE, DATE, DATE, DAT..."


In [28]:
print(my_df.shape)

(1, 5)


**Task 10: Show the unique set of elements from each column in the data frame**

In [29]:
# Use set() on the data in each column to show the list of unique elements
# set(my_df.iloc[:, 1])

**Task 11: Show your data frame**

In [30]:
# Type the name of your data frame on a line by itself to display it
my_df

,Verbs,Subjects,Objects,Entity,Ent_Type
0,"[is, played, earned, drafted, played, born, na...","[Collison, Collison, He, Collison, who, He, Co...","[seasons, honors, American, recruit, MVP, stea...","[Darren, Michael, Collison, August, 23, ,, 198...","[PERSON, PERSON, PERSON, DATE, DATE, DATE, DAT..."


Don't forget to also process your second article in the same fashion as you did for the first one. 

In [31]:
# Code for processing the second article starts here

# First article was done in fashison as shown in labs. 
# I was however unable to get the df to look correct and behaved more like a list of lists. Also wasnt able to complete taks 10

# Second article was done with assistance from Gemini

In [32]:
scraped_data2 = urllib.request.urlopen(req2)

article2 = scraped_data2.read() # Extract the data from the response object
parsed_article2 = bs.BeautifulSoup(article2,'lxml') # Use lxml as the back end parser
paragraphs = parsed_article2.find_all('p')

article_text = ""
for p in paragraphs:
    article_text += p.text

len(article_text)

clean_text = re.sub(r'\[[0-9]*\]', '', article_text)
cleaner_text = re.sub( r'\s+', ' ', clean_text)

nlp = spacy.load("en_core_web_sm")
my_article2 = nlp(cleaner_text)

len(my_article2) # Length in tokens

7463

In [33]:
my_spans = list(my_article2.sents)

rows = []

def get_subject(token):
    for child in token.children:
        if child.dep_ in ("nsubj"):
            return child.text
    return "N/A"

def get_object(token):
    for child in token.children:
        if child.dep_ in ("dobj"):
            return child.text
    return "N/A"

for idx, span in enumerate(my_spans):
    root = span.root

    subject = get_subject(root)
    obj = get_object(root)

    ents = [(ent.text, ent.label_) for ent in span.ents]
    if len(ents) == 0:
        ent_text = "N/A"
        ent_type = "N/A"
    else:
        ent_text = ", ".join([e[0] for e in ents])
        ent_type = ", ".join([e[1] for e in ents])

    rows.append({
        "span_index": idx,
        "root": root.text if root else "N/A",
        "subject": subject,
        "object": obj,
        "entity": ent_text,
        "entity_type": ent_type
    })

df = pd.DataFrame(rows)
df = df.fillna("N/A")
df


,span_index,root,subject,object,entity,entity_type
0,0,was,Abbott,N/A,"Darrell Lance Abbott, August 20, 1966, Decembe...","ORG, DATE, DATE, PERSON, NORP"
1,1,was,He,N/A,"Pantera, Damageplan, Vinnie Paul","ORG, PERSON, PERSON"
2,2,considered,N/A,N/A,N/A,N/A
3,3,began,son,N/A,"Jerry Abbott, Abbott, age 12, Pantera, 1983, 16","PERSON, GPE, DATE, ORG, DATE, DATE"
4,4,went,Abbott,N/A,"Abbott, Diamond Darrell","GPE, PERSON"
...,...,...,...,...,...,...
299,299,placed,Abbott,N/A,"2015, Abbott, 5, Gibson, The Top 10 Metal Guit...","DATE, GPE, CARDINAL, PERSON, WORK_OF_ART"
300,300,said,Erickson,N/A,"Anne Erickson, Abbott","PERSON, PERSON"
301,301,remembered,N/A,N/A,N/A,N/A
302,302,praised,N/A,playing,"2017, Ernie Ball String Theory, Sevenfold, Zac...","DATE, PERSON, GPE, PERSON, PERSON"


In [34]:
len(set(df.iloc[:, 2]))

100

In [35]:
my_spans = list(my_article.sents)

rows = []

def get_subject(token):
    for child in token.children:
        if child.dep_ in ("nsubj"):
            return child.text
    return "N/A"

def get_object(token):
    for child in token.children:
        if child.dep_ in ("dobj"):
            return child.text
    return "N/A"

for idx, span in enumerate(my_spans):
    root = span.root

    subject = get_subject(root)
    obj = get_object(root)

    ents = [(ent.text, ent.label_) for ent in span.ents]
    if len(ents) == 0:
        ent_text = "N/A"
        ent_type = "N/A"
    else:
        ent_text = ", ".join([e[0] for e in ents])
        ent_type = ", ".join([e[1] for e in ents])

    rows.append({
        "span_index": idx,
        "root": root.text if root else "N/A",
        "subject": subject,
        "object": obj,
        "entity": ent_text,
        "entity_type": ent_type
    })

df1 = pd.DataFrame(rows)
df1 = df1.fillna("N/A")
df1


,span_index,root,subject,object,entity,entity_type
0,0,is,Collison,N/A,"Darren Michael Collison, August 23, 1987, Amer...","PERSON, DATE, NORP, ORG, ORG"
1,1,played,Collison,seasons,"Collison, four seasons","PERSON, DATE"
2,2,earned,He,honors,"three, the Frances Pomeroy Naismith Award, his...","CARDINAL, ORG, DATE, QUANTITY, CARDINAL"
3,3,drafted,N/A,N/A,"the New Orleans Hornets, first, 21st, 2009, NBA","ORG, ORDINAL, ORDINAL, DATE, ORG"
4,4,played,Collison,N/A,"Collison, the Indiana Pacers, Dallas Mavericks...","ORG, ORG, ORG, ORG, ORG, ORG"
5,5,born,N/A,N/A,"Collison, Rancho Cucamonga, California, Dennis...","PERSON, GPE, GPE, PERSON, PERSON, GPE"
6,6,named,N/A,N/A,"Etiwanda High School, Dave Kleckner, Collison,...","ORG, PERSON, PERSON, ORDINAL"
7,7,listed,N/A,N/A,"four, Rivals.com, Collison, 16, 100, 2005","CARDINAL, ORG, ORG, CARDINAL, CARDINAL, DATE"
8,8,was,He,N/A,"Jordan Farmar, Bruins, 2005–06 season, the fol...","PERSON, PERSON, DATE, DATE"
9,9,awarded,N/A,MVP,"MVP, the Maui Invitational Tournament, Decembe...","ORG, ORG, DATE, DATE, DATE"


In [36]:
len(set(df1.iloc[:, 2]))

11

Don't forget to put in a concluding paragraph with your observations comparing the two articles.

In [37]:
# The two articles are rather similar in some ways but very different in others. One of the similarities is the length with both having ~7000 tokens.
# The number of sentences however is different with article 1 being made up of only 58 sentences compared to article 2 which has 304.
# This could be due to fact that the subject of article 2 is a musician that palyed in several bands with many other famous musicians. 
# This would more sentences with more subjects that is necessary to tell their complete story. The subject of article 1 however was a NBA player, 
# who is also relatively less famous. The sentences from his wikipedia page primarily were about him so could be longer and only had 11 unique subjects.